# Import necessary packages

In [1]:
import requests
from bs4 import BeautifulSoup
import re


# Fetch all books from website

The url is http://books.toscrape.com/

In [2]:
site = "http://books.toscrape.com/"
page = requests.get(site)
soup = BeautifulSoup(page.content, 'html.parser')
my_books = {}
anchor_tags = [a['href'] for a in soup.find_all('a')]
urls = anchor_tags[4:53]
genre = None
pattern = r'/([^/_]+)_\d+'  

for url in urls:
    page_number = 1
    genre_books = {}
    match = re.search(pattern, url)
    if match:
        genre = match.group(1)
        genre = genre.replace('-',' ')
    genre_url = site + url
    while True:
      genre_page = requests.get(genre_url)
      book_soup = BeautifulSoup(genre_page.content, 'html.parser')
      for product in book_soup.find_all('article', {'class': 'product_pod'}):
        title = product.h3.a['title'].lower()
        availability = product.select_one('p.availability').text.strip().lower()
        genre_books[title] = availability
      my_books[genre] = genre_books
      next_link = soup.find("li", {"class": "next"}).find("a")
      if ("404" not in genre_page.text):
        page_number += 1
        next_url = site + url.replace('index.html', f'page-{page_number}.html')
        genre_url = next_url
        # print(f"Found next page: {next_url}")
      else:
        break

  
  
        

# Helper function to check for availability

In [3]:

def in_stock(title: str, topic: str) -> bool:
  genre = topic.lower()
  title = title.lower()
  found = False
  if genre in my_books.keys():
    if title in my_books[genre].keys():
      if my_books[genre][title] == 'in stock':
        found = True
  return found
     

# Some test cases

In [4]:
in_stock("While You Were Mine", "Historical Fiction")


True

In [5]:
in_stock("Online Marketing for Busy Authors: A Step-By-Step guide", "Self help")


True

In [6]:
in_stock("The MooSEwood Cookbook: Recipes from Moosewood Restaurant, Ithaca, New York", "food and driNk")


True

In [7]:
in_stock("While You Were Mine", "Science")


False

# Helper function to see books in a specified genre

In [8]:

def see_books(topic):
  genre = topic.lower()
  return my_books[genre]
     

# Test Cases

In [9]:
see_books("Science")

{"the most perfect thing: inside (and outside) a bird's egg": 'in stock',
 'immunity: how elie metchnikoff changed the course of modern medicine': 'in stock',
 'sorting the beef from the bull: the science of food fraud forensics': 'in stock',
 'tipping point for planet earth: how close are we to the edge?': 'in stock',
 'the fabric of the cosmos: space, time, and the texture of reality': 'in stock',
 'diary of a citizen scientist: chasing tiger beetles and other new ways of engaging the world': 'in stock',
 'the origin of species': 'in stock',
 'the grand design': 'in stock',
 'peak: secrets from the new science of expertise': 'in stock',
 'the elegant universe: superstrings, hidden dimensions, and the quest for the ultimate theory': 'in stock',
 'the disappearing spoon: and other true tales of madness, love, and the history of the world from the periodic table of the elements': 'in stock',
 "surely you're joking, mr. feynman!: adventures of a curious character": 'in stock',
 'seven br